In [1]:
#import libraries for task
import numpy as np
import matplotlib.pyplot as plt

import lab_funcs as lf
import lab_funcs_disp as lfd
import my_funcs as mf

# Вычисление рядов

## Khan-sum

### Задания

1. Объясните различие в погрешностях при различных порядках суммирования.
3. Почему алгорит Кэхэна имеет значительно лучшую точность, чем последовательное суммирование?
3. Получим ли мы те же значения погрешностей, если будем суммировать последовательность со слагаемыми разных знаков? Проверьте на следующей последовательности:
$$x_k=\sin k.$$
4. Что произойдет с погрешностью, если элементы выборки с разными знаками упорядочить по возрастанию? По возрастанию абсолютной величины? Проверьте экспериментально.


### Решение

1. Если складывать суммы не учитывая порядок, то мы получим суммы где складываются большие числа с маленькими и погрешность увеличивается за счет отбрасывания нижних порядков.
Если складывать поочередно, то погрешность при каждом суммирование будет меньше, т.к. мы будем складывать относительно равные числа (равного порядка) и погрешность не будет такой большой как в первом случае, т.к. нижние порядки учитваются.
Продемострируем это на практике.

In [2]:
#generate data
K = 7
perm_data = lf.samples(K)
sort_data = lf.sort_samples(K)
sum_perm = lf.direct_sum(perm_data)
sum_sort = lf.direct_sum(sort_data)
cur_sum = lf.exact_sum(K)

print("Relative permut data error: ", lf.relative_error(cur_sum, sum_perm))
print("Relative sort data error: ", lf.relative_error(cur_sum, sum_sort))


Relative permut data error:  4.6550208134767723e-11
Relative sort data error:  1.9908519277181958e-11


2. Алгоритм Кэхэна не отбрасывает остаток, он его сохраняет на каждом шаге суммирования. И результат следующего суммирования будет брать во внимание остаток от предыдущего суммирования и создавать остаток для нового суммирования.
При суммирование по возрастанию мы уменьшили погрешность, но не учитывали остаток при суммировании.

Получим сумму для алгоритма Кэхэна

In [3]:
kahan_sum = lf.Kahan_sum(perm_data)
print("Relative Kahan sum error: ", lf.relative_error(cur_sum, kahan_sum))

Relative Kahan sum error:  0.0


3. При суммирование знакочередующегося ряда мы получаем погрешность близкую к машинной. Для алгоритма Кэхэна ничего принципиально не изменится если будем суммировать слагаемые с разными знаками.

In [8]:
N = 1000
correct_sin_sum = mf.get_correct_sin_sum(N)
sin_perm_data = mf.get_sin_perm_samples(N)
sum_sin_perm = mf.Kahan_sum(sin_perm_data)

print("Relative sum error to Kahan permut sin data: ", lf.relative_error(correct_sin_sum, sum_sin_perm))

Relative sum error to Kahan permut sin data:  9.54772862162757e-16


4. Если упорядочивать элементы выборки, то относительная погрешность будет уменьшатся. Лучшим вариантом упорядочивания является $\rightarrow$ упорядочивание не по абсолютной величине.

In [10]:

correct_sin_sum = mf.get_correct_sin_sum(N)
sin_sort_data = mf.get_sin_sort_samples(N)
sin_abs_sort_data = mf.get_sin_abs_sort_samples(N)

sum_sin_sort = mf.Kahan_sum(sin_sort_data)
sum_abs_sin_sort = mf.Kahan_sum(sin_abs_sort_data)

print("Relative sum error to Kahan permut sin data: ",
       lf.relative_error(correct_sin_sum, sum_sin_perm))
print("Relative sum error to Kahan sort sin data: ",
       lf.relative_error(correct_sin_sum, sum_sin_sort))
print("Relative sum error to Kahan abs sort sin data: ",
       lf.relative_error(correct_sin_sum, sum_abs_sin_sort))

Relative sum error to Kahan permut sin data:  9.54772862162757e-16
Relative sum error to Kahan sort sin data:  4.0918836949832464e-16
Relative sum error to Kahan abs sort sin data:  5.455844926644328e-16


## Вычисление дисперсии

### Задания
1. Обьясните, почему формулы оценки дисперсии имеют разные погрешности, хотя чтобы их применить, нужно выполнить одни и те же действия, но в разном порядке? Оцените погрешности обоих формул.
2. Предложите однопроходную формулу для оценки мат. ожидания и дисперсии, основанную на первой формуле для дисперсии. Воспользуйтесь компенсационным суммированием, чтобы увеличить точность. Попробуйте увеличить точность вычисления по сравнению со второй формулой хотя бы на два порядка.

### Решение
1. 